<a href="https://colab.research.google.com/github/eaNODE/wimpbo/blob/master/na.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://martin-thoma.com/solving-linear-equations-with-gaussian-elimination/
# https://martin-thoma.com/polynomial-interpolation/
# https://stackoverflow.com/questions/31957096/gaussian-elimination-with-pivoting-in-python
# http://drsfenner.org/blog/2016/03/givens-rotations-and-qr/
# https://rosettacode.org/wiki/QR_decomposition
# https://www.quantstart.com/articles/QR-Decomposition-with-Python-and-NumPy
# https://nicoguaro.github.io/posts/numerical-12/
# https://www.math.ubc.ca/~pwalls/math-python
# https://integratedmlai.com/find-the-determinant-of-a-matrix-with-pure-python-without-numpy-or-scipy/
# https://www.unioviedo.es/compnum/labs/PYTHON/Interpolation.html
# https://www.math.ubc.ca/~pwalls/math-python/linear-algebra/applications/
# https://proofwiki.org/wiki/Vandermonde_Determinant
# https://mmas.github.io/interpolation-scipy


# TODO : complete my cooperation with matinfar for his idea using nc doc in my google drive

from __future__ import print_function
import math
import math, sys
import scipy
import scipy.linalg
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# ---------SCIPY-&-NUMPY---------
print("=======================================AX=B USING LU===================================")
A = scipy.array([ [7, 3, -1, 2], [3, 8, 1, -4], [-1, 1, 4, -1], [2, -4, -1, 6] ])
P, L, U = scipy.linalg.lu(A)
print("\n---p---")
print(P)
print("\n---l---")
print(L)
print("\n---u---")
print(U)
A = np.array([[2., 1., 1.], [1., 3., 2.], [1., 0., 0.]])
B = np.array([4., 5., 6.])
LU = scipy.linalg.lu_factor(A)
x = scipy.linalg.lu_solve(LU, B)
print("\n---[x] using LU---")
print(x)
print("\n-----[x] using A inv-----")
print(np.linalg.inv(A) @ B)

print("=======================================VANDERMONDE===================================")
# points >>>> (0,6) , (3,1) , (8,2)
print("\n---using column_stack---")
x = np.array([0,3,8])
x = np.column_stack([[1,1,1],x,x**2])
print(x)
print("------")
y = np.array([6, 1, 2]).reshape(3,1)
print(y)
print("\n---SOLVE EQUATION---")
a = scipy.linalg.solve(x,y)
print(a)
print("\n---VANDERMONDE DET---")
x = np.array([0,3,8])
print(np.vander(x, increasing=True))
print(np.linalg.det(np.vander(x)))
print("====================================GAUSSIAN ELIMINATION WITH PIVOT ALGO======================================")

# ---------------GEWP------------------

def gaussian_elimination_with_pivot(m):
  # forward elimination
    n = len(m)
    for i in range(n):
        pivot(m, n, i)
        for j in range(i+1, n):
            m[j] = [m[j][k] - m[i][k]*m[j][i]/m[i][i] for k in range(n+1)]

    if m[n-1][n-1] == 0: raise ValueError('[!] No unique solution')

    # backward substitution
    x = [0] * n
    for i in range(n-1, -1, -1):
        s = sum(m[i][j] * x[j] for j in range(i, n))
        x[i] = (m[i][n] - s) / m[i][i]
    return x

def pivot(m, n, i):
    max_row = max(range(i, n), key=lambda r: abs(m[r][i]))
    m[i], m[max_row] = m[max_row], m[i]


if __name__ == '__main__':
    m = [[0,-2,6,-10], [-1,3,-6,5], [4,-12,8,12]]
    #   m = [[1,-1,3,2], [3,-3,1,-1], [1,1,0,3]]
    #   m = [[1,-1,3,2], [6,-6,2,-2], [1,1,0,3]]
    print(gaussian_elimination_with_pivot(m))
    print("====================================LU DECOMPO ALGO======================================")

print("\n---solve linear equation l..u..p---")

import pprint

def mult_matrix(M, N):
                                                                                                                                                                                                      
    tuple_N = zip(*N)                                                                                                                                                                                    
    return [[sum(el_m * el_n for el_m, el_n in zip(row_m, col_n)) for col_n in N] for row_m in M]

def pivot_matrix(M):

    m = len(M)                                                                                                                                                                                      
    id_mat = [[float(i ==j) for i in range(m)] for j in range(m)]                                                                                                                                                                                             
    for j in range(m):
        row = max(range(j, m), key=lambda i: abs(M[i][j]))
        if j != row:                                                                                                                                                                                                                          
            id_mat[j], id_mat[row] = id_mat[row], id_mat[j]
    return id_mat

def lu_decomposition(A):

    n = len(A)                                                                                                                                                                                                                 
    L = [[0.0] * n for i in range(n)]
    U = [[0.0] * n for i in range(n)]
                                                                                                                                                                                            
    P = pivot_matrix(A)
    PA = mult_matrix(P, A)
                                                                                                                                                                                                                     
    for j in range(n):

        L[j][j] = 1.0
        for i in range(j+1):
            s1 = sum(U[k][j] * L[i][k] for k in range(i))
            U[i][j] = PA[i][j] - s1
                                                                                                                                                               
        for i in range(j, n):
            s2 = sum(U[k][j] * L[i][k] for k in range(j))
            L[i][j] = (PA[i][j] - s2) / U[j][j]

    return (P, L, U)

# AX = B
# LUX = B
# LY = B

A = [[7, 3, -1, 2], [3, 8, 1, -4], [-1, 1, 4, -1], [2, -4, -1, 6]]
P, L, U = lu_decomposition(A)

npl = np.asarray(L)
npu = np.asarray(U)
A = np.matmul(npl, npu)
# X = np.matmul(B, np.linalg.inv(np.matmul(npl, npu)))

print("A:")
pprint.pprint(A)

print("------------\n")
print("P:")
pprint.pprint(P)

print("------------\n")
print("L:")
print(np.asarray(L))

print("------------\n")
print("U:")
print(np.asarray(U))



print("\n---solve linear equation l..u---")

def lu(mat):

    n = len(mat)
    lower = [[0 for x in range(n)] for y in range(n)]
    upper = [[0 for x in range(n)] for y in range(n)]


    for i in range(n):
        # UPPER
        for k in range(i, n):
            # Summation of L(i, j) * U(j, k)
            sum = 0
            for j in range(i):
                sum += (lower[i][j] * upper[j][k])
            # Evaluating U(i, k)
            upper[i][k] = mat[i][k] - sum

        # LOWER
        for k in range(i, n):
            if (i == k):
                lower[i][i] = 1 # Diagonal as 1
            else:
                # Summation of L(k, j) * U(j, i)
                sum = 0
                for j in range(i):
                    sum += (lower[k][j] * upper[j][i])
                #  Evaluating L(k, i)
                lower[k][i] = int((mat[k][i] - sum) / upper[i][i])

    return lower, upper

A = [[2, -1, -2],
       [-4, 6, 3],
       [-4, -2, 8]]

l, u  = lu(A)
npl = np.asarray(l)
npu = np.asarray(u)
A = np.matmul(npl, npu)
# X = np.matmul(B, np.linalg.inv(np.matmul(npl, npu)))

print("L : ")
print("")
print(npl)
print("")
print("U : ")
print("")
print(npu)
print("A : ")
print("")
print(A)

# AX = b
# LUX = b
# LY = b

print("=================================ROOT FINDING ALGO=========================================")

print("\n---regula falsi---")

MAX_ITER = 1000000

# The function is x^3 - x^2 + 2 
def func( x ): 
    return (x * x * x - x * x + 2) 
  
# Prints root of func(x) in interval [a, b] 
def regulaFalsi( a , b): 
    if func(a) * func(b) >= 0: 
        print("Must have opposite sign!") 
        return -1
      
    c = a # Initialize result
      
    for i in range(MAX_ITER): 
          
        # Find the point that touches x axis 
        c = (a * func(b) - b * func(a))/ (func(b) - func(a)) 
          
        # Check if the above found point is root 
        if func(c) == 0: 
            break
          
        # Decide the side to repeat the steps 
        elif func(c) * func(a) < 0: 
            b = c 
        else: 
            a = c 
    print("The value of root is : " , '%.4f' %c) 
  

a =-200
b = 300
regulaFalsi(a, b) 

print("\n---newton---")

def newton(f,Df,x0,epsilon,max_iter):
    xn = x0
    for n in range(0,max_iter):
        fxn = f(xn)
        if abs(fxn) < epsilon:
            print('Found solution after',n,'iterations.')
            return xn
        Dfxn = Df(xn)
        if Dfxn == 0:
            print('Zero derivative. No solution found.')
            return None
        xn = xn - fxn/Dfxn
    print('Exceeded maximum iterations. No solution found.')
    return None

f = lambda x: x**3 - x**2 - 1
Df = lambda x: 3*x**2 - 2*x
approx = newton(f,Df,1,1e-10,10)
print(approx)


print("\n---bisection---")

def bisection(f,a,b,N):
    if f(a)*f(b) >= 0:
        print("Bisection method fails.")
        return None
    a_n = a
    b_n = b
    for n in range(1,N+1):
        m_n = (a_n + b_n)/2
        f_m_n = f(m_n)
        if f(a_n)*f_m_n < 0:
            a_n = a_n
            b_n = m_n
        elif f(b_n)*f_m_n < 0:
            a_n = m_n
            b_n = b_n
        elif f_m_n == 0:
            print("Found exact solution.")
            return m_n
        else:
            print("Bisection method fails.")
            return None
    return (a_n + b_n)/2
  
f = lambda x: x**2 - x - 1
approx_phi = bisection(f,1,2,25)
print("[+] the root is ", approx_phi)

error_bound = 2**(-26)
print("[+] err is ", error_bound)

# cal err .... abs(xreal - approx_phi) <= b-a/math.pow(2, N+1)

print("[+] abs of diff real x and xn is less than err ", abs( (1 + 5**0.5)/2 - approx_phi) < error_bound)

print("=================================INTERPOLATION ALGO=========================================")

print("\n---vandermonde matrix---")
def vander_mat(x):
    # x is a list of points
    n = len(x)
    van = np.zeros((n,n))
    power = np.array(range(n))
    for row in range(n):
        van[row, :] = x[row]**power
    return van

def det_vander_mat(v):
    pass

print(vander_mat([1,2,3,4]))


print("\n---lagrange---")

# f(x) = x^3
x = np.array([0, 1, 2])
y = x**3
poly = lagrange(x, y)
print(poly)
print(Polynomial(poly).coef)

x_parabola = np.array([0, 20, 0]) # array for x
y_parabola = np.array([0, 10, 50]) # array for y
plt.figure()
u = plt.plot(x_parabola,y_parabola,'ro') # plot the points
t = np.linspace(0, 1, len(x_parabola)) # parameter t to parametrize x and y
pxLagrange = scipy.interpolate.lagrange(t, x_parabola) # X(T)
pyLagrange = scipy.interpolate.lagrange(t, y_parabola) # Y(T)
n = 100
ts = np.linspace(t[0],t[-1],n)
xLagrange = pxLagrange(ts) # lagrange x coordinates
yLagrange = pyLagrange(ts) # lagrange y coordinates
plt.plot(xLagrange, yLagrange,'b-',label = "Polynomial")
plt.show()


'''
For each  k=0,1,…,n , there exists a unique polynomial ℓk  of degree ≤ n such as ℓk(xj)=δkj  (one if  k=j , zero otherwise):

ℓk(z)=(z−x0)…(z−xk−1)(z−xk+1)…(z−xn)/(xk−x0)…(xk−xk−1)(xk−xk+1)…(xk−xn).
 
The polynomials  ℓ0,ℓ1,…,ℓn  are the Lagrange fundamental polynomials.

The Lagrange polynomial for the points  x0,x1,…,xn  with values  y0,y1,…,yn  is

Pn(x)=y0ℓ0(x)+y1ℓ1(x)+⋯+ynℓn(x)
'''

# globals to control some behavior
func_type = "tanh"   # can be sine or tanh
points = "variable"  # can be variable or fixed

npts = 15

def fun_exact(x):
    """ the exact function that we sample to get the points to
    interpolate through """

    if func_type == "sine":
        return np.sin(x)
    elif func_type == "tanh":
        return 0.5*(1.0+np.tanh((x-1.0)/0.1))


def get_interp_points(N, xmin, xmax):
    """ get the x points that we interpolate at """
    if points == "fixed":
        x = np.linspace(xmin, xmax, N)

    elif points == "variable":
        # the Chebyshev nodes
        x = 0.5*(xmin + xmax) + \
            0.5*(xmax - xmin)*np.cos(2.0*np.arange(N)*math.pi/(2*N))

    return x
    
    
def lagrange_poly(x, xp, fp):
    """ given points (xp, fp), fit a lagrange polynomial and return
        the value at point x """

    f = 0.0
    
    # sum over points
    for m in range(len(xp)):

        # create the Lagrange basis polynomial for point m        
        l = None

        for n in range(len(xp)):
            if n == m:
                continue
            
            if l == None:
                l = (x - xp[n])/(xp[m] - xp[n])
            else:
                l *= (x - xp[n])/(xp[m] - xp[n])
                
        f += fp[m]*l

    return f


if func_type == "sine":
    xmin = 0.0
    xmax = 2.0*math.pi
elif func_type == "tanh":
    xmin = 0.0
    xmax = 2.0




# xp, fp are the points that we build the interpolant from
xp = get_interp_points(npts, xmin, xmax)
fp = fun_exact(xp)


# xx are the finely grided data that we will interpolate at to get
# the interpolated function values ff
xx = np.linspace(xmin, xmax, 200)
ff = np.zeros(len(xx))

for n in range(len(xx)):
    ff[n] = lagrange_poly(xx[n], xp, fp)

# exact function values at the interpolated points
fexact = fun_exact(xx)


# error
e = fexact-ff


plt.subplot(211)
plt.scatter(xp, fp, marker="x", color="r", s=30)
plt.plot(xx, ff, color="k")
plt.plot(xx, fexact, color="0.5")
plt.xlim(xmin, xmax)
plt.subplot(212)
plt.plot(xx, e)
plt.xlim(xmin, xmax)
plt.savefig("lagrange.png")


  
print("\n---newtons divided differece---")  

x = np.array([-1, 0, 1, 2]) # x coordinates in space
y = np.array([1, 1, 2, 0]) # f(x)

def getNDDCoeffs(x, y):
    """ Creates NDD pyramid and extracts coeffs """
    n = np.shape(y)[0]
    pyramid = np.zeros([n, n]) # Create a square matrix to hold pyramid
    pyramid[::,0] = y # first column is y
    for j in range(1,n):
        for i in range(n-j):
            # create pyramid by updating other columns
            pyramid[i][j] = (pyramid[i+1][j-1] - pyramid[i][j-1]) / (x[i+j] - x[i])
    return pyramid[0] # return first row

coeff_vector = getNDDCoeffs(x, y)
print(coeff_vector)

# create as many polynomials as size of coeff_vector
final_pol = np.polynomial.Polynomial([0.]) # our target polynomial
n = coeff_vector.shape[0] # get number of coeffs
for i in range(n):
    p = np.polynomial.Polynomial([1.]) # create a dummy polynomial
    for j in range(i):
        # each vector has degree of i
        # their terms are dependant on 'x' values
        p_temp = np.polynomial.Polynomial([-x[j], 1.]) # (x - x_j)
        p = np.polymul(p, p_temp) # multiply dummy with expression
    p *= coeff_vector[i] # apply coefficient
    final_pol = np.polyadd(final_pol, p) # add to target polynomial

p = np.flip(final_pol[0].coef, axis=0)
print(p)

# Evaluate polynomial at X axis and plot result
x_axis = np.linspace(0, 10, num=5000)
y_axis = np.polyval(p, x_axis)

plt.plot(x_axis, y_axis)
plt.savefig("nnd.png")
